In [5]:
import requests
import folium
import pandas as pd
import wget
from sklearn.cluster import KMeans
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bSoup
import numpy as np
import re
from geopy.geocoders import Nominatim as geocoder
import csv
from pandas import json_normalize

print('Library import')
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text 
soup = bSoup(data,"html5lib") 


contents = []
table = soup.find('table')
for row in table.findAll('td'):
    col = {}
    #Skip cells of table with not assinged.
    if row.span.text == 'Not assigned':
        pass
    #Split the contents of table cell, where first 3 chars are the postal code, and borough is contained outside the (),
    #and the neighbourhood is within the ().
    else:
        col['PCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(col)
tblCont = pd.DataFrame(contents)
tblCont['Borough'] = tblCont['Borough'].replace({'EtobicokeNorthwest':'Etobicoke Northwest',
                                            'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                                            'East YorkEast Toronto':'East York/East Toronto'})


Library import


In [8]:
#Csv file from the url contains the long and lat of Toronto, just have to parse out the ones that match the scraped postal codes.
longLatCSV ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
parsePCodes = pd.read_csv(longLatCSV)

In [12]:
#Join the parsePCodes with tblCont on postal codes, and remove added columns from the dataframe that have NaN data,
#delete duplicate data, and reset the index.
finalizedLatLong = pd.merge(tblCont, parsePCodes, left_on = 'PCode', right_on = 'Postal Code')
finalizedLatLong = finalizedLatLong.drop('Postal Code', axis = 1)
finalizedLatLong = finalizedLatLong.drop_duplicates()
finalizedLatLong.index = range(len(finalizedLatLong))
finalizedLatLong.head()

,PCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [13]:
#Foursquare api info
lim = 50
ver = '20220409'
c_id = 'TPIL203UWTGQRQCXIVERR4XCXJOYE0PC5GRVKYAFGA2JWPQD'
c_srct_id = 'M5LN5VA54524SARZLVUCE0VXEZHHSNQ2AURODZBW3PEQJHLL'
couProvCity = 'Toronto, Ontario, Canada'

In [14]:
#Use foursquare agent
gLoc = geocoder(user_agent = "AI-Proj-Key-2022")
cityLoc = gLoc.geocode(couProvCity)
latitude = cityLoc.latitude
longitude = cityLoc.longitude
tlongitude = longitude
tlatitude = latitude

mapOfRegion = folium.Map(location = [latitude, longitude], zoom_start = 11)
mapOfRegion

In [17]:
#Place markers on neighbourhoods that have been scraped.
for lati, longi, borough, neighbourhood in zip(finalizedLatLong['Latitude'], finalizedLatLong['Longitude'],finalizedLatLong['Borough'],finalizedLatLong['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lati, longi],
    fill = True,
    fill_opacity = 0.1,
    color = '#cc8131',
    fill_color = 'yellow',
    radius = 6,
    popup = label,
    parse_html = False).add_to(mapOfRegion)
mapOfRegion

In [19]:
#Reading the needed csv files to datasets to join on common columns.
dataCrime = pd.read_csv ('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
dataNeighProfiles = pd.read_csv ('neighbourhood-profiles-2016-csv.csv')
dataNeigh = pd.read_csv ('Neighbourhoods-4326.csv')

In [20]:
#Joining dataNeigh with dataCrime on the same Neighbourhood name and AREA_DESC name.
crimeAndNeigh = dataCrime.merge(dataNeigh, how = 'inner', left_on = 'Neighbourhood', right_on = 'AREA_DESC')
crimeAndNeigh

,OBJECTID_x,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area_x,Shape__Length_x,AREA_ATTR_ID,AREA_DESC,AREA_ID,AREA_LONG_CODE,AREA_NAME,AREA_SHORT_CODE,CLASSIFICATION,CLASSIFICATION_CODE,LATITUDE,LONGITUDE,OBJECTID_y,PARENT_AREA_ID,Shape__Area_y,Shape__Length_y,X,Y,_id,geometry
0,1,Yonge-St.Clair,97,12528,20,29,39,27,34,37,31.0,0.09,295.3,2,3,7,2,6,6,4.3,0.00,47.9,37,20,12,19,24,28,23.3,0.17,223.5,0,0,0,0,0,0,0.0,0.00,0.0,6,5,6,8,5,4,5.7,-0.20,31.9,4,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582,26005523,Yonge-St.Clair,2480143,97,Yonge-St.Clair (97),97,NaN,NaN,NaN,NaN,17547249,NaN,2.222464e+06,8130.411276,NaN,NaN,135,"{'type': 'Polygon', 'coordinates': (((-79.3911..."
1,2,York University Heights,27,27593,271,296,361,344,357,370,333.2,0.04,1340.9,105,100,105,92,92,144,106.3,0.57,521.9,107,139,98,105,122,108,113.2,-0.11,391.4,1,0,2,1,1,0,0.8,-1.00,0.0,59,84,70,75,88,79,75.8,-0.10,286.3,30,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326,26005526,York University Heights,2480146,27,York University Heights (27),27,Neighbourhood Improvement Area,NIA,NaN,NaN,17547201,NaN,2.541821e+07,25632.335240,NaN,NaN,132,"{'type': 'Polygon', 'coordinates': (((-79.5052..."
2,3,Lansing-Westgate,38,16164,44,80,68,85,75,72,70.7,-0.04,445.4,19,22,27,26,16,32,23.7,1.00,198.0,34,27,41,42,50,39,38.8,-0.22,241.3,0,0,0,0,10,0,1.7,-1.00,0.0,11,5,9,17,35,11,14.7,-0.69,68.1,4,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625,26005527,Lansing-Westgate,2480147,38,Lansing-Westgate (38),38,NaN,NaN,NaN,NaN,17547185,NaN,1.025436e+07,15391.584160,NaN,NaN,131,"{'type': 'Polygon', 'coordinates': (((-79.4399..."
3,4,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,160.2,0.19,1411.8,63,53,41,52,63,61,55.5,-0.03,412.1,51,57,66,58,64,84,63.3,0.31,567.4,1,1,1,1,2,1,1.2,-0.50,6.8,23,21,24,35,44,42,31.5,-0.05,283.7,23,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920,26005485,Yorkdale-Glen Park,2480105,31,Yorkdale-Glen Park (31),31,Emerging Neighbourhood,EN,NaN,NaN,17546209,NaN,1.156669e+07,13953.408100,NaN,NaN,70,"{'type': 'Polygon', 'coordinates': (((-79.4396..."
4,5,Stonegate-Queensway,16,25051,88,71,76,95,87,82,83.2,-0.06,327.3,34,29,12,32,31,34,28.7,0.10,135.7,71,45,49,49,39,64,52.8,0.64,255.5,0,0,0,0,0,0,0.0,0.00,0.0,21,14,16,26,25,22,20.7,-0.12,87.8,7,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878,26005451,Stonegate-Queensway,2480071,16,Stonegate-Queensway (16),16,NaN,NaN,NaN,NaN,17547137,NaN,1.518135e+07,16379.806320,NaN,NaN,128,"{'type': 'Polygon', 'coordinates': (((-79.4926..."
5,6,Tam O'Shanter-Sullivan,118,27446,106,90,98,124,133,107,109.7,-0.20,389.9,22,19,22,18,32,25,23.0,-0.22,91.1,49,68,61,54,77,67,62.7,-0.13,244.1,0,1,0,0,1,2,0.7,1.00,7.3,28,39,24,27,18,30,27.7,0.67,109.3,4,6,9,8,3,3,5.5,0.00,10.9,5.422345e+06,10750.468125,26005452,Tam O'Shanter-Sullivan,2480072,118,Tam O'Shanter-Sullivan (118),118,NaN,NaN,NaN,NaN,17547121,NaN,1.040854e+07,14891.488940,NaN,NaN,127,"{'type': 'Polygon', 'coordinates': (((-79.3197..."
6,7,The Beaches,63,21567,83,108,86,111,94,81,93.8,-0.14,375.6,8,23,7,10,30,20,16.3,-0.33,92.7,44,42,77,33,43,57,49.3,0.33,264.3,0,0,0,0,0,0,0.0,0.00,0.0,15,14,19,48,10,16,20.3,0.60,74.2,8,5,3,7,5,9,6.2,0.80,41.7,3.595829e

#### Transpose the neighbourhood profiles dataset as row has the neighbourhoods not the columns like the other datasets, and join to single dataset.

In [21]:
#Transposing 
dataNeighProfilesTransposed = dataNeighProfiles.T
crimeProfilesNeigh = dataNeighProfilesTransposed.merge(crimeAndNeigh, how = 'inner', left_on = 0, right_on = 'Neighbourhood')